In [ ]:
%load_ext autoreload
%autoreload 2

import sys, os, time, datetime

import psutil
process = psutil.Process(os.getpid())
print(f"Available memory: {int(psutil.virtual_memory().available / 1024 / 1024)} Mb")

# nbytes = process.memory_info().rss # bytes
# print(f"Process memory: {nbytes} bytes ({nbytes / 1024 / 1024} Mb)") 

import tensorflow as tf
from tensorflow import keras
print(f"Using Tensorflow v{tf.__version__}")

# For managing relative imports from notebook
if '..' not in sys.path: sys.path.append('..')

import config.config as dfc
import deepfake.dfutillib as df
import deepfake.modelutil as mutil

In [ ]:
# ------------------------- Model Architecture -------------------------

def create_encdec_network():
#{
    model = None

    # Layer #0: Input
    frame_input = keras.layers.Input(shape=(360, 640, 3))

    # Layer #1: First set of convolutional layers
    X = keras.layers.Conv2D(filters=32, kernel_size=(11,11), strides=(2,4))(frame_input)
    X = keras.layers.MaxPooling2D()(X)
    
    # Layer #2: Second set of convolutional layers
    X = keras.layers.Conv2D(filters=64, kernel_size=(3,3), strides=(1,1))(X)
    X = keras.layers.MaxPooling2D()(X)
    
    # Layer #3: Third set of convolutional layers
    X = keras.layers.Conv2D(filters=128, kernel_size=(3,3), strides=(1,1))(X)
    X = keras.layers.MaxPooling2D()(X)
    
    # Layer #4-5: Fully connected layers
    X_flat = keras.layers.Flatten()(X)
    X = keras.layers.Dense(720, activation = "relu")(X_flat)
    X = keras.layers.Dense(720, activation = "relu")(X)
    X = keras.layers.Concatenate(axis=1)([X, X_flat])
    X = keras.layers.Reshape((20,18,130))(X)
    
    # Layer #6: First set of deconvolutional layers
    X = keras.layers.UpSampling2D()(X)
    X = keras.layers.Conv2DTranspose(filters=64, kernel_size=(3,3), strides=(1,1))(X)

    # Layer #7: Second set of deconvolutional layers
    X = keras.layers.UpSampling2D()(X)
    X = keras.layers.Conv2DTranspose(filters=32, kernel_size=(3,3), strides=(1,1))(X)

    # Layer #8: Last deconvolution to diff image output
    X = keras.layers.UpSampling2D()(X)
    diff_output = keras.layers.Conv2DTranspose(filters=3, kernel_size=(11,11), strides=(2,4))(X)
        
    # Layer #9-10: Final fully connected to binary REAL/FAKE class
    X = keras.layers.MaxPooling2D(pool_size=(6, 6))(diff_output)
    X = keras.layers.Flatten()(X)
    X = keras.layers.Dense(512, activation = "relu")(X)
    X = keras.layers.Dense(512, activation = "relu")(X)

    # Output layers
    dflat_output = keras.layers.Flatten(name='dflat_output')(diff_output)
    fake_output = keras.layers.Dense(1, activation = "sigmoid", name='fake_output')(X)
    
    model = keras.models.Model(inputs = frame_input, outputs = [dflat_output, fake_output])
    model.summary()

    return model
#}

In [ ]:
# ------------------------- Model Instantiation -------------------------

model = create_encdec_network()
model.compile(optimizer=keras.optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, decay=1e-6), 
              loss_weights=[1.0, 0.2], metrics=['accuracy', 'binary_accuracy'],
              loss={'dflat_output': tf.keras.losses.MeanSquaredError(), 
                    'fake_output': tf.keras.losses.BinaryCrossentropy()})

In [ ]:
# ------------------------- Loaders Disk -> Model -------------------------
    
# Model data loaders
trloader = mutil.ModelLoader(split='train')
vdloader = mutil.ModelLoader(split='validate')

# Data shapes
inshape, labelshape = (None, 360, 640, 3), (None, 1)
imgoutshape = (None, dfc.TARGETSZ[0]*dfc.TARGETSZ[1]*3)

# TF Dataset wrappers for data loaders
trdataset = tf.data.Dataset.from_generator(generator=trloader.lazy_loader, 
    output_types=(tf.uint8, {'dflat_output': tf.uint8, 'fake_output': tf.uint8}),
    output_shapes=(inshape, {'dflat_output': imgoutshape, 'fake_output': labelshape}))

vddataset = tf.data.Dataset.from_generator(generator=vdloader.lazy_loader, 
    output_types=(tf.uint8, {'dflat_output': tf.uint8, 'fake_output': tf.uint8}),
    output_shapes=(inshape, {'dflat_output': imgoutshape, 'fake_output': labelshape}))

# ------------------------- Callbacks Model -> Disk/Db -------------------------

# Timestamped logging/model directories
outdir = datetime.datetime.now().strftime('%Y%m%d-%H%M%S')
tboarddir = f"{dfc.TBOARD_LOG}/{outdir}"
modeldir = f'{dfc.MODEL_STORE}/{outdir}'
if not os.path.isdir(tboarddir): os.mkdir(tboarddir)
if not os.path.isdir(modeldir): os.mkdir(modeldir)

# Custom callback
epwrapdb_cb = mutil.PgdbWrapupCb(model)

# TF built-in callbacks
tboard_cb = tf.keras.callbacks.TensorBoard(
    log_dir=tboarddir, histogram_freq=1, write_graph=True, 
    write_images=True, update_freq=60, profile_batch=7)

savemodel_cb = tf.keras.callbacks.ModelCheckpoint(
    filepath=(f'{modeldir}/' + 'model.{epoch:02d}-{val_loss:.2f}.hdf5'), 
    monitor='val_loss', save_best_only=False, save_freq='epoch')


In [ ]:
# ------------------------- Training -------------------------

history = model.fit(
    x=trdataset, steps_per_epoch=trloader.epochsz,
    validation_data=vddataset, validation_steps=vdloader.epochsz,
    callbacks=[tboard_cb, savemodel_cb, epwrapdb_cb],
    initial_epoch=0, verbose=2, epochs=1000)